# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9684660be0>
├── 'a' --> tensor([[ 1.3906, -0.7060,  1.5600],
│                   [ 1.3181, -1.0069, -1.0808]])
└── 'x' --> <FastTreeValue 0x7f9684660bb0>
    └── 'c' --> tensor([[ 1.4424,  0.4607, -0.8687, -0.1770],
                        [ 0.4459,  0.4218,  0.1039,  0.3253],
                        [-0.4717, -0.0106, -1.9308,  0.3745]])

In [4]:
t.a

tensor([[ 1.3906, -0.7060,  1.5600],
        [ 1.3181, -1.0069, -1.0808]])

In [5]:
%timeit t.a

65.3 ns ± 1.05 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9684660be0>
├── 'a' --> tensor([[-0.9602,  0.1450,  2.1498],
│                   [-0.5997,  0.8573,  0.7193]])
└── 'x' --> <FastTreeValue 0x7f9684660bb0>
    └── 'c' --> tensor([[ 1.4424,  0.4607, -0.8687, -0.1770],
                        [ 0.4459,  0.4218,  0.1039,  0.3253],
                        [-0.4717, -0.0106, -1.9308,  0.3745]])

In [7]:
%timeit t.a = new_value

70.7 ns ± 1.67 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.3906, -0.7060,  1.5600],
               [ 1.3181, -1.0069, -1.0808]]),
    x: Batch(
           c: tensor([[ 1.4424,  0.4607, -0.8687, -0.1770],
                      [ 0.4459,  0.4218,  0.1039,  0.3253],
                      [-0.4717, -0.0106, -1.9308,  0.3745]]),
       ),
)

In [10]:
b.a

tensor([[ 1.3906, -0.7060,  1.5600],
        [ 1.3181, -1.0069, -1.0808]])

In [11]:
%timeit b.a

60 ns ± 2 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 2.6850,  0.6207,  0.0541],
               [ 1.6581,  0.7785, -2.1379]]),
    x: Batch(
           c: tensor([[ 1.4424,  0.4607, -0.8687, -0.1770],
                      [ 0.4459,  0.4218,  0.1039,  0.3253],
                      [-0.4717, -0.0106, -1.9308,  0.3745]]),
       ),
)

In [13]:
%timeit b.a = new_value

581 ns ± 9.83 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

922 ns ± 8.29 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.9 µs ± 120 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

281 µs ± 8.56 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

250 µs ± 7.68 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f95ce965df0>
├── 'a' --> tensor([[[ 1.3906, -0.7060,  1.5600],
│                    [ 1.3181, -1.0069, -1.0808]],
│           
│                   [[ 1.3906, -0.7060,  1.5600],
│                    [ 1.3181, -1.0069, -1.0808]],
│           
│                   [[ 1.3906, -0.7060,  1.5600],
│                    [ 1.3181, -1.0069, -1.0808]],
│           
│                   [[ 1.3906, -0.7060,  1.5600],
│                    [ 1.3181, -1.0069, -1.0808]],
│           
│                   [[ 1.3906, -0.7060,  1.5600],
│                    [ 1.3181, -1.0069, -1.0808]],
│           
│                   [[ 1.3906, -0.7060,  1.5600],
│                    [ 1.3181, -1.0069, -1.0808]],
│           
│                   [[ 1.3906, -0.7060,  1.5600],
│                    [ 1.3181, -1.0069, -1.0808]],
│           
│                   [[ 1.3906, -0.7060,  1.5600],
│                    [ 1.3181, -1.0069, -1.0808]]])
└── 'x' --> <FastTreeValue 0x7f95ce965af0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

43.5 µs ± 662 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f95d5558f10>
├── 'a' --> tensor([[ 1.3906, -0.7060,  1.5600],
│                   [ 1.3181, -1.0069, -1.0808],
│                   [ 1.3906, -0.7060,  1.5600],
│                   [ 1.3181, -1.0069, -1.0808],
│                   [ 1.3906, -0.7060,  1.5600],
│                   [ 1.3181, -1.0069, -1.0808],
│                   [ 1.3906, -0.7060,  1.5600],
│                   [ 1.3181, -1.0069, -1.0808],
│                   [ 1.3906, -0.7060,  1.5600],
│                   [ 1.3181, -1.0069, -1.0808],
│                   [ 1.3906, -0.7060,  1.5600],
│                   [ 1.3181, -1.0069, -1.0808],
│                   [ 1.3906, -0.7060,  1.5600],
│                   [ 1.3181, -1.0069, -1.0808],
│                   [ 1.3906, -0.7060,  1.5600],
│                   [ 1.3181, -1.0069, -1.0808]])
└── 'x' --> <FastTreeValue 0x7f95cd90df40>
    └── 'c' --> tensor([[ 1.4424,  0.4607, -0.8687, -0.1770],
                        [ 0.4459,  0.4218,  0.1039,  0.3253],
                 

In [23]:
%timeit t_cat(trees)

40.8 µs ± 420 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

72.8 µs ± 750 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.3906, -0.7060,  1.5600],
                [ 1.3181, -1.0069, -1.0808]],
       
               [[ 1.3906, -0.7060,  1.5600],
                [ 1.3181, -1.0069, -1.0808]],
       
               [[ 1.3906, -0.7060,  1.5600],
                [ 1.3181, -1.0069, -1.0808]],
       
               [[ 1.3906, -0.7060,  1.5600],
                [ 1.3181, -1.0069, -1.0808]],
       
               [[ 1.3906, -0.7060,  1.5600],
                [ 1.3181, -1.0069, -1.0808]],
       
               [[ 1.3906, -0.7060,  1.5600],
                [ 1.3181, -1.0069, -1.0808]],
       
               [[ 1.3906, -0.7060,  1.5600],
                [ 1.3181, -1.0069, -1.0808]],
       
               [[ 1.3906, -0.7060,  1.5600],
                [ 1.3181, -1.0069, -1.0808]]]),
    x: Batch(
           c: tensor([[[ 1.4424,  0.4607, -0.8687, -0.1770],
                       [ 0.4459,  0.4218,  0.1039,  0.3253],
                       [-0.4717, -0.0106, -1.9308,  0.3745]],
         

In [26]:
%timeit Batch.stack(batches)

95.1 µs ± 1.56 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.3906, -0.7060,  1.5600],
               [ 1.3181, -1.0069, -1.0808],
               [ 1.3906, -0.7060,  1.5600],
               [ 1.3181, -1.0069, -1.0808],
               [ 1.3906, -0.7060,  1.5600],
               [ 1.3181, -1.0069, -1.0808],
               [ 1.3906, -0.7060,  1.5600],
               [ 1.3181, -1.0069, -1.0808],
               [ 1.3906, -0.7060,  1.5600],
               [ 1.3181, -1.0069, -1.0808],
               [ 1.3906, -0.7060,  1.5600],
               [ 1.3181, -1.0069, -1.0808],
               [ 1.3906, -0.7060,  1.5600],
               [ 1.3181, -1.0069, -1.0808],
               [ 1.3906, -0.7060,  1.5600],
               [ 1.3181, -1.0069, -1.0808]]),
    x: Batch(
           c: tensor([[ 1.4424,  0.4607, -0.8687, -0.1770],
                      [ 0.4459,  0.4218,  0.1039,  0.3253],
                      [-0.4717, -0.0106, -1.9308,  0.3745],
                      [ 1.4424,  0.4607, -0.8687, -0.1770],
                      [ 0.4459,  

In [28]:
%timeit Batch.cat(batches)

176 µs ± 4.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

561 µs ± 13.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
